# Portfolio optimization accelerator

<img src="https://i.pinimg.com/originals/24/a7/61/24a761a5fd24fa28cf9ff1111a95f122.jpg" alt="Source: https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.pinterest.com%2Fpin%2F91549804901025206%2F&psig=AOvVaw2763c4gGmjK2xPgw02jrak&ust=1668513426289000&source=images&cd=vfe&ved=0CA8QjRxqFwoTCPDh5oLPrfsCFQAAAAAdAAAAABAE" width="50%"/>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cvxpy import *

### Data Import

put csv of monthly stock prices for:

Microsoft
Visa
Walmart

in Colab default temp folder

In [ ]:
mp = pd.read_csv("monthly_prices.csv",index_col=0) 
mr = pd.DataFrame()

In [ ]:
mp.head()

### Plot the data

In [ ]:
plt.figure(figsize=(10,5))
plt.plot([i for i in range(1,25)],mp['MSFT'],lw=3,marker='o',markersize=12)
plt.plot([i for i in range(1,25)],mp['V'],lw=3,c='red',marker='^',markersize=12)
plt.plot([i for i in range(1,25)],mp['WMT'],lw=3,marker='*',markersize=12)
plt.legend(mp.columns,fontsize=16)
plt.xlabel("Months",fontsize=18)
plt.ylabel("Stock price (Monthly average)",fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.show()

### Compute monthly returns

In [ ]:
# compute monthly returns
for s in mp.columns:
    date = mp.index[0]
    pr0 = mp[s][date] 
    for t in range(1,len(mp.index)):
        date = mp.index[t]
        pr1 = mp[s][date]
        ret = (pr1-pr0)/pr0
        mr.at[date,s] = ret # NB set_value deprecated, now using .at[]. See https://stackoverflow.com/questions/60294463/attributeerror-dataframe-object-has-no-attribute-set-value
        pr0 = pr1

In [ ]:
mr.head()

### Get symbol names

In [ ]:
# get symbol names
symbols = mr.columns

### Convert monthly return data frame to a numpy matrix

In [ ]:
return_data = mr.values.T # as_matrix() deprecated, using .values. See https://stackoverflow.com/questions/61102281/dataframe-object-has-no-attribute-as-matrix

In [ ]:
plt.figure(figsize=(10,5))
plt.plot([i for i in range(1,24)],100*mr['MSFT'],lw=3,marker='o',markersize=12)
plt.plot([i for i in range(1,24)],100*mr['V'],lw=3,c='red',marker='^',markersize=12)
plt.plot([i for i in range(1,24)],100*mr['WMT'],lw=3,marker='*',markersize=12)
plt.legend(mp.columns,fontsize=16)
plt.xlabel("Months",fontsize=18)
plt.ylabel("Monthly return (%))",fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.show()

### Mean return

In [ ]:
r = np.asarray(np.mean(return_data, axis=1))

### Covariance matrix

In [ ]:
C = np.asmatrix(np.cov(return_data))

In [ ]:
C

### Print expected returns and risk

In [ ]:
for j in range(len(symbols)):
    print ('%s: Exp ret = %f, Risk = %f' %(symbols[j],r[j], C[j,j]**0.5))

### Set up the optimization model

In [ ]:
# Number of variables
n = len(symbols)

# The variables vector
x = Variable(n)

# The minimum return
req_return = 0.02

# The return
ret = r.T*x

# The risk in xT.Q.x format
risk = quad_form(x, C)

# The core problem definition with the Problem class from CVXPY
prob = Problem(Minimize(risk), [sum(x)==1, ret >= req_return, x >= 0])

### Try solving the problem (within a `try/except` loop)

In [ ]:
try:
    prob.solve()
    print ("Optimal portfolio")
    print ("----------------------")
    for s in range(len(symbols)):
       print (" Investment in {} : {}% of the portfolio".format(symbols[s],round(100*x.value[s],2)))
    print ("----------------------")
    print ("Exp ret = {}%".format(round(100*ret.value,2)))
    print ("Expected risk    = {}%".format(round(100*risk.value**0.5,2)))
except:
    print ("Error")

In [ ]:
prob.status

In [ ]:
x.value

Source References: 


https://www.kdnuggets.com/2019/06/optimization-python-money-risk.html

https://github.com/tirthajyoti/Optimization-Python